In [1]:
# title: Decorator sample
# change log: 2021-08-22 add memoization

# package:

# reference:
# 

In [2]:
import time

def sum(a, b):
    # 開始測量
    start = time.localtime()
    print(time.strftime("%Y-%m-%d %H:%M:%S", start))

    # 開始執行
    print("開始執行 sum")
    result= a + b

    # 輸出結果
    print("執行時間：%f 秒" % (time.time() - time.mktime(start)))
    return result

print('sum 執行結果', sum(1, 2))


2024-09-20 17:35:28
開始執行 sum
執行時間：0.025480 秒
sum 執行結果 3


In [3]:
import time
def sum(a, b):
    return a + b

def debug(func):
    def wrapper(*args, **kwargs):
        # 開始測量
        start = time.localtime()
        print(time.strftime("%Y-%m-%d %H:%M:%S", start))

        print('開始執行 ', func.__name__)
        result = func(*args, **kwargs)

        # 輸出結果
        print("執行時間：%f 秒" % (time.time() - time.mktime(start)))
        print(func.__name__, '執行結果', result)
    return wrapper

# 可以簡化寫法
# debug(sum)(1, 2)
debug_sum = debug(sum)
debug_sum(1, 2)


2024-09-20 17:35:28
開始執行  sum
執行時間：0.036479 秒
sum 執行結果 3


In [4]:
import time
from functools import wraps

def debug(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # 開始測量
        start = time.localtime()
        print(time.strftime("%Y-%m-%d %H:%M:%S", start))

        print('開始執行 ', func.__name__)
        result = func(*args, **kwargs)

        # 輸出結果
        print("執行時間：%f 秒" % (time.time() - time.mktime(start)))
        print(func.__name__, '執行結果', result)
    return wrapper

@debug
def sum(a, b):
    return a + b

sum(1, 2)

# 不加 @wraps 會發生以下問題
# print('func', sum.__name__)
# print('doc', sum.__doc__)
# func名稱變成wrapper
# func wrapper
# doc None

2024-09-20 17:35:28
開始執行  sum
執行時間：0.052479 秒
sum 執行結果 3
